In [1]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thuner.data as data
import thuner.data.dispatch as dispatch
import thuner.grid as grid
import thuner.option as option
import thuner.track as track
import thuner.analyze as analyze
import thuner.parallel as parallel
import thuner.visualize as visualize
import thuner.log as log

notebook_name = "gridrad_demo.ipynb"

ModuleNotFoundError: No module named 'thuner'

['/home/ewan/miniconda/envs/THUNER/lib/python310.zip',
 '/home/ewan/miniconda/envs/THUNER/lib/python3.10',
 '/home/ewan/miniconda/envs/THUNER/lib/python3.10/lib-dynload',
 '',
 '/home/ewan/.local/lib/python3.10/site-packages',
 '/home/ewan/miniconda/envs/THUNER/lib/python3.10/site-packages',
 '/home/ewan/Documents/THUNER/thuner']

In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
start = "2010-01-20T22:00:00"
end = "2010-01-21T02:00:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/gridrad_demo"
event_start

'2010-01-20'

In [3]:
# if output_parent.exists():
# shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create and save the dataset options
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.option.GridRadSevereOptions(**times_dict, **gridrad_dict)
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.option.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.option.ERA5Options(**times_dict, **era5_dict)
data_options = data.option.DataOptions(
    datasets=[gridrad_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")
gridrad_options = data_options.dataset_by_name("gridrad")

# Create and save the grid_options dictionary 
grid_options = grid.GridOptions(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
# track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
# track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
# visualize_options = {
#     obj: visualize.option.runtime_options(obj, save=True, style="presentation")
#     for obj in ["mcs"]
# }
visualize_options = None

2024-12-11 16:23:48,846 - thuner.data.option - INFO - Generating era5 filepaths.
2024-12-11 16:23:48,848 - thuner.data.option - INFO - Generating era5 filepaths.
2024-12-11 16:23:48,851 - thuner.data.option - INFO - Generating era5 filepaths.
2024-12-11 16:23:48,869 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:23:48,871 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.


In [4]:
times = data.utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options.model_copy(), grid_options.copy()]
args += [track_options.model_copy(), visualize_options]
track.simultaneous_track(*args, output_directory=output_parent)

2024-12-11 16:23:50,460 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo.
2024-12-11 16:23:50,462 - thuner.track - INFO - Beginning simultaneous tracking.
2024-12-11 16:23:51,662 - thuner.track - INFO - Processing 2010-01-20T22:00:00.
2024-12-11 16:23:51,663 - thuner.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T22:00:00.
2024-12-11 16:23:51,664 - thuner.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T220000Z.nc
2024-12-11 16:23:53,433 - thuner.track - INFO - Processing hierarchy level 0.
2024-12-11 16:23:53,434 - thuner.track - INFO - Tracking convective.
2024-12-11 16:24:06,629 - thuner.track - INFO - Tracking middle.
2024-12-11 16:24:07,008 - thuner.track - INFO - Tracking anvil.
2024-12-11 16:24:07,375 - thuner.track - INFO - Processing hierarchy level 1.
2024-12-11 16:24:07,376 - thuner.track - INFO - Tracking mcs.
2024-12-11 16:24:08,050 - thuner.match.match - INFO - Matching mcs o

KeyboardInterrupt: 

In [5]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 4
kwargs = {"initializer": parallel.initialize_process, "processes": num_processes}
with log.logging_listener(), multiprocessing.get_context("spawn").Pool(**kwargs) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.model_copy(), grid_options.copy()]
        args += [track_options.model_copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-12-11 16:24:29,179 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_0.
2024-12-11 16:24:29,179 - thuner.track - INFO - Beginning simultaneous tracking.
2024-12-11 16:24:29,221 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_1.
2024-12-11 16:24:29,223 - thuner.track - INFO - Beginning simultaneous tracking.
2024-12-11 16:24:29,321 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_2.
2024-12-11 16:24:29,322 - thuner.track - INFO - Beginning simultaneous tracking.
2024-12-11 16:24:29,377 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_3.
2024-12-11 16:24:29,378 - thuner.track - INFO - Beginning simultaneous tracking.
2024-12-11 16:24:30,275 - thuner.track - INFO - Processing 2010-01-20T22:00:00.
2024-12-11 16:24:30,275 - th

In [6]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-12-11 16:45:17,791 - thuner.parallel - INFO - Stitching all attribute, mask and record files.
2024-12-11 16:45:17,807 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:17,809 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-12-11 16:45:21,423 - thuner.parallel - INFO - Stitching record files.
2024-12-11 16:45:21,456 - thuner.parallel - INFO - Stitching attribute files.
2024-12-11 16:45:23,049 - thuner.parallel - INFO - Stitching mask files.


In [7]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent, analysis_options)

2024-12-11 16:45:26,116 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:26,117 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-12-11 16:45:26,299 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:26,300 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.


convective_contained  anvil_contained  \
time                universal_id                                          
2010-01-20 22:00:00 1                             True            False   
                    2                             True             True   
                    3                             True             True   
                    4                             True             True   
                    5                             True             True   
...                                                ...              ...   
2010-01-21 01:50:00 11                            True             True   
                    14                            True             True   
                    15                           False            False   
                    27                            True             True   
                    30                            True             True   

                                  initially_contained  velocity  area  offset  \
time                universal_id                                                
2010-01-20 22:00:00 1                           False      True  True    True   
                    2                            True      True  True    True   
                    3                            True      True  True   False   
                    4                            True      True  True    True   
                    5                            True      True  True    True   
...                                               ...       ...   ...     ...   
2010-01-21 01:50:00 11                           True      True  True    True   
                    14                           True      True  True   False   
                    15                           True      True  True    True   
                    27                           True      True  True   False   
                    30                           True      True  True   False   

                                  major_axis  axis_ratio  duration  parents  \
time                universal_id                                              
2010-01-20 22:00:00 1                   True        True      True    False   
                    2                   True        True     False    False   
                    3                   True        True     False    False   
                    4                   True       False      True     True   
                    5                   True        True     False    False   
...                                      ...         ...       ...      ...   
2010-01-21 01:50:00 11                  True        True      True     True   
                    14                  True        True      True     True   
                    15                 False       False      True     True   
                    27                  True       False      True    False   
                    30                  True       False     False    False   

                                  children  
time                universal_id            
2010-01-20 22:00:00 1                 True  
                    2                 True  
                    3                False  
                    4                 True  
                    5                 True  
...                                    ...  
2010-01-21 01:50:00 11                True  
                    14               False  
                    15                True  
                    27               False  
                    30               False  

[216 rows x 11 columns]

In [8]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
figure_options = visualize.option.horizontal_attribute_options(
    figure_name, style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

2024-12-11 16:45:28,429 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:28,430 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-12-11 16:45:28,716 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:00:00.000000000.
2024-12-11 16:45:30,359 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:30,360 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-12-11 16:45:33,629 - thuner.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T22:00:00.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-12-11 16:45:44,077 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:10:00.000000000.
2024-12-11 16:45:44,155 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:20:00.000000000.
2024-12-11 16:45:44,397 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:30:00.000000000.
2024-12-11 16:45:46,333 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:40:00.000000000.
2024-12-11 16:45:47,801 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:47,801 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-12-11 16:45:47,843 - thuner.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-12-11 16:45:47,843 - thuner.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-12-11 16:45:48,322 - thuner.grid - WARNING - altitude_spacing not specified. Will attem